In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers



In [2]:
import pandas as pd
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, pipeline
from datasets import Dataset
from google.colab import drive
from huggingface_hub import login
from tqdm import tqdm

from google.colab import drive



In [3]:


data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/orientation-cz-train.tsv", sep="\t")



class_0 = data[data['label'] == 0]
class_1 = data[data['label'] == 1]

subset_size_class_0 = int(0.01 /8* len(class_0))
subset_size_class_1 = int(0.01 /8* len(class_1))

resampled_class_0 = resample(class_0, n_samples=subset_size_class_0, random_state=42)
resampled_class_1 = resample(class_1, n_samples=subset_size_class_1, random_state=42)

resampled_data = pd.concat([resampled_class_0, resampled_class_1]).sample(frac=1, random_state=42).reset_index(drop=True)


dataset = Dataset.from_pandas(resampled_data)


model_name = "bigscience/bloom-560m"
generator = pipeline("text-generation", model=model_name, tokenizer=model_name, device=0)
def classify_batch(batch):
    results = []
    for text in tqdm(batch["text"], desc="Classifying texts"):
        prompt = (
            f"Imagine you are a political analyst. Your task is to determine the political leaning of a given statement. "
            f"Each statement belongs to one of two categories: 'left-leaning' or 'right-leaning'. "
            f"Respond with the political leaning as clearly as possible. Use the following examples to guide your decision:\n\n"
            f"Examples:\n"
            f"1. 'The government should implement policies to reduce wealth inequality.' => left-leaning\n"
            f"2. 'Cutting taxes for high-income earners stimulates investment and job creation.' => right-leaning\n"
            f"3. 'Promoting universal access to education benefits society as a whole.' => left-leaning\n"
            f"4. 'Privatizing public services increases efficiency and innovation.' => right-leaning\n\n"
            f"Here is the statement:\n"
            f"Statement: {text}\n"
            f"Answer:"
        )
        output = generator(prompt, max_new_tokens=20, temperature=0.7, num_return_sequences=1, do_sample=True)
        raw_prediction = output[0]["generated_text"].strip()

        prediction = raw_prediction.split("Answer:")[1].strip().lower()

        if "left-leaning" in prediction.lower():
            results.append(0)  # Assuming 0 = left-leaning
        else:
            results.append(1)  # Assuming 1 = right-leaning


    return {"predictions": results}



dataset = dataset.map(classify_batch, batched=True, batch_size=8)





print(classification_report(dataset["label"], dataset["predictions"]))



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cpu


Map:   0%|          | 0/5 [00:00<?, ? examples/s]


Classifying texts: 100%|██████████| 5/5 [04:28<00:00, 53.74s/it]

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.60      1.00      0.75         3

    accuracy                           0.60         5
   macro avg       0.30      0.50      0.38         5
weighted avg       0.36      0.60      0.45         5




/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
